In [1]:
import re
import numpy as np
import pickle
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [2]:
labels = {"negative" : 0, "positive" : 1}

def get_model(file_path):
    try:
       with open(file_path, "rb") as file:
            data = pickle.load(file)
    except Exception as e:
        print(f"Cannot load model file due to {e}")
        
    classifier_model = data['classifier']
    embeddings_model = data['embeddings']

    return classifier_model, embeddings_model


def text_processor(text):
    text = text.lower()

    # Removing urls 
    text = re.sub(r'(https?://\S+|www\.\S+)', "", text)
    # Removing HTML Tags
    text = re.sub(r"<.*?>", "", text)
    # Puncuation and special Char
    text = re.sub(r"[^\w\d\s]", " ", text)
    # Remove spaces
    text = text.strip()

    # Stopwords
    text = [word for word in text.split() if word not in ENGLISH_STOP_WORDS]
    text = " ".join(text)
    
    # Lemmatization
    text = " ".join([lemmatizer.lemmatize(w) for w in text.split()])

    # Removing single letter from sentence
    text = re.sub(r"\s\w{1}\s", "", text)

    return text

def vectorize( tokens, model):
    vectors = [
        model[word] for word in tokens if word in model
    ]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)

    return np.mean(vectors, axis =0)

In [3]:
def pred_pipe(input_text, model_path):
    classifier_model, embeddings_model = get_model(model_path)

    process_text = text_processor(input_text)
    embeddings = vectorize(process_text.split(), embeddings_model.wv)

    expected = classifier_model.n_features_in_
    if embeddings.shape[0] != expected:
        raise ValueError(f"Feature mismatch: got {embeddings.shape[0]}, model expects {expected}")

    predict = classifier_model.predict([embeddings])
    sentiment = [key for key, value in labels.items() if value == predict[0]]
    return sentiment[0].capitalize()


In [4]:
MODEL_PATH = "sentiment_classifier.pkl"
USER_INPUT = "Fantastic, held my attention the whole way through, jokes were so funny and natural. Felt like the old muppets. Loved the celebrity guests, especially Maya Rudolph. The music was also good and I loved the final act. This is my first review on here, I had to finally leave one for the sake of the Muppets. Bring back the show!!!"

sentiment = pred_pipe(USER_INPUT, MODEL_PATH)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.8.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
sentiment

'Positive'